# Gemicai tutorial 2. From data pre-processing to model evaluation
In this tutorial we'll be developing a neural network to classify the orientation of a mammography image. To get a good understanding of the Gemicai library, this tutorial describes the full process of doing so, from data pre-processing to model evaluation. The tutorial will be structured as follows:

[2.0 Preliminaries](#2.0)<br>
[2.1 Data preproccessing](#2.0)<br>
[2.2 Classifier initialization](#2.0)<br>
[2.3 Hyper parameter optimisation](#2.0)<br>
[2.4 Model evaluation](#2.0)

## 2.0 Preliminaries
<a id='2.0'></a>
Some text

In [13]:
import gemicai as gem
import torch
import torchvision.models as models

In [10]:
relevant_dicom_attributes = ['BodyPartExamined', 'StudyDescription', 'SeriesDescription']

train_dataset = '/mnt/SharedStor/train_dataset/MG/'
eval_dataset = '/mnt/SharedStor/eval_dataset/MG/'
hmm_dataset = '/mnt/SharedStor/dataset/MG/'

resnet18 = models.resnet18(pretrained=True)



# net = gem.Classifier(resnet18, dataset.classes('BodyPartExamined'), enable_cuda=False)

In [11]:
dataset = gem.DicomoDataset.get_dicomo_dataset(train_dataset, labels=relevant_dicom_attributes)
dataset.summarize('StudyDescription')
dataset = gem.DicomoDataset.get_dicomo_dataset(eval_dataset, labels=relevant_dicom_attributes)
dataset.summarize('StudyDescription')
dataset = gem.DicomoDataset.get_dicomo_dataset(hmm_dataset, labels=relevant_dicom_attributes)
dataset.summarize('StudyDescription')

| Class                              |   Frequency |
|------------------------------------+-------------|
| Mammografie SVOB beiderzijds       |          28 |
| Mammografie beiderzijds            |         256 |
| Mammopunctie stereotactisch links  |          54 |
| Mammografie mammapoli beiderzijds  |         196 |
| Copie SVOB                         |          93 |
| Mammografie follow up rechts       |          51 |
| Echo lokalisatie mammatumor links  |           4 |
| Mammografie na lokalisatie links   |           6 |
| Mammografie follow up bdz          |         254 |
| Mammografie follow up links        |          37 |
| Preparaat                          |          19 |
| Mammografie na lokalisatie rechts  |          16 |
| Mammopunctie stereotactisch rechts |          23 |
| Mammografie mammapoli rechts       |           3 |
| Mammografie links                  |           6 |
| Mammografie rechts                 |           7 |
| Echo mamma beiderzijds             |        